In [1]:
import pandas as pd
import numpy as np
import datetime
import copy
import json
import jenkspy

In [2]:
data = pd.read_csv('../scratch/avif_dump.csv', encoding='utf-8', low_memory=False)

In [3]:
data.drop_duplicates(inplace=True)

In [4]:
#data = data[data.Complete == 1]

In [5]:
data.dropna(subset=['ySquareX', 'ySquareY', 'SpeciesId'], inplace=True)

In [6]:
data.SpeciesId = data.SpeciesId.astype(int).astype(str)

In [7]:
def scaleWeek(val): # čtyřtýdenní "měsíce"
    OldRange = (52 - 1)
    NewRange = (48 - 1)
    return int((((val - 1) * NewRange) / OldRange) + 1)

In [8]:
data['Week'] = data.ObsDate.apply(lambda x: scaleWeek(int(datetime.datetime.strptime(x, "%Y-%m-%d" ).strftime("%V"))))

In [9]:
data['Square'] = data.ySquareY.astype(int).astype(str) + data.ySquareX.astype(int).astype(str)

In [10]:
# dump species list
with open('../speclist.js', 'w', encoding='utf-8') as f:
    f.write('var specList = ' 
            + json.dumps(data[['SpeciesId', 'SpeciesSci', 'Species']].set_index('SpeciesId').to_dict(orient='index')) 
            + ';')

In [11]:
out = {}
for sq in data.Square.unique():
    if sq not in out:
        out[sq] = {}
    for sp in data.SpeciesId.unique():
        out[sq][sp] = [0] * 48
    out[sq]['wks'] = [0] * 48

In [12]:
for sqr in out:
    sub = data[data.Square == sqr]
    for w in sub.groupby(['Week']).ObsListId.unique().reset_index().values:
        out[sqr]['wks'][w[0] - 1] = len(w[1])
        
    for specie in list(sub.SpeciesId.unique()):
        for w in sub[sub.SpeciesId == specie].groupby(['Week']).ObsListId.unique().reset_index().values:
            out[sqr][specie][w[0] - 1] = len(w[1])

In [13]:
# podil druhu na listech
for sqr in out:
    for key in out[sqr]:
        if key == 'wks':
            continue
        out[sqr][key] = list(np.divide(out[sqr][key], out[sqr]['wks']))

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [14]:
# likvidace nanu a nekonecen
def cln(val):
    if np.isfinite(val) == True:
        return val
    else:
        return 0
    
clean = np.vectorize(cln, otypes=[np.float])

for sqr in out.values():
    for key in sqr:
        sqr[key] = list(clean(sqr[key]))

In [15]:
#del prazdnych klicu
out_cp = copy.deepcopy(out)
for sqr in out_cp:
    for key in out_cp[sqr]:
        if key == 'wks':
            continue
        if sum(out_cp[sqr][key]) == 0:
            del out[sqr][key]

In [16]:
for sq in out:
    tmp = out[sq]
    for t in tmp: # Numpy JSON vohack
        tmp[t] = list(map(float, tmp[t]))
    with open('../stats/' + str(sq) + '.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(tmp))

In [17]:
stats = {'sum': 0}
for sq in out:
    stats[sq] = sum(out[sq]['wks'])
    stats['sum'] += sum(out[sq]['wks'])

In [18]:
with open('../stats/stats.js', 'w', encoding='utf-8') as f:
    f.write('var stats = ' + json.dumps(stats) + ';')

In [19]:
o = []
for s in list(stats.values())[1:]:
    o.append((s / stats['sum']))

In [20]:
breaks = jenkspy.jenks_breaks(o, nb_class=5)

In [21]:
breaks

[1.6751541141785044e-05,
 0.0012898686679174483,
 0.0038193513803269903,
 0.008844813722862504,
 0.017404851246314662,
 0.03454167783436076]